https://docs.opencv.org/4.8.0/db/d95/classcv_1_1ORB.html

In [ ]:
%pip install nibabel numpy matplotlib opencv-contrib-python scikit-image

In [ ]:
from __future__ import print_function
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
from threading import Thread
import math

https://stackoverflow.com/questions/61833375/loading-a-nifti-through-nibabel-and-using-the-shape-function

In [ ]:
def get_slice(nii_data, slice):
  return nii_data[:,:,slice]

In [ ]:
def open_nii(path):
  my_img  = nib.load(path)
  nii_data = my_img.get_fdata()
  return nii_data

In [ ]:
def show_slice(nii_data, slice):
  plt.imshow(get_slice(nii_data, slice))
  plt.show()

In [ ]:
def show(img):
  plt.imshow(img)
  plt.show()

In [ ]:
def show_all_slices(nii_data):
  for slice in range(nii_data.shape[2]):
    plt.imshow(get_slice(nii_data,slice))
    plt.show()

In [ ]:
def show_with_label(img, label):
  plt.figure(figsize = (50,7))
  plt.subplot(1,2,1)
  plt.imshow(img, 'gray', interpolation='none')
  plt.imshow(label, 'jet', interpolation='none', alpha=0.1)
  plt.show()

In [ ]:
def cv_convert(img):
  return cv.normalize(img, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)

In [ ]:
def find_keypoints(img, nfeatures=500, scaleFactor=1.2, nlevels=8, edgeThreshold=31, firstLevel=0, WTA_K=2, scoreType=cv.ORB_HARRIS_SCORE, patchSize=31, fastThreshold=20):
  orb = cv.ORB_create(nfeatures=nfeatures, scaleFactor=scaleFactor, nlevels=nlevels, edgeThreshold=edgeThreshold, firstLevel=firstLevel, WTA_K=WTA_K, scoreType=scoreType, patchSize=patchSize, fastThreshold=fastThreshold)
  kp = orb.detect(img,None)
  return kp

In [ ]:
def is_point_in_mask(mask, x, y):
    # Check if the point is within the mask boundaries
    if 0 <= x < mask.shape[1] and 0 <= y < mask.shape[0]:
        # Check if the pixel value at the specified coordinates is non-zero
        return mask[y, x] == 2 # 0 is background, 1 is pancreas, 2 is cancer
    else:
        # Point is outside the mask boundaries
        return False


In [ ]:
def split_list(alist, wanted_parts=1):
  length = len(alist)
  return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
            for i in range(wanted_parts) ]

In [ ]:
def algo_run(data, points_in_mask, kp_len, correct_slices, matrix, i, nfeatures=500, scaleFactor=1.2, nlevels=8, edgeThreshold=31, firstLevel=0, WTA_K=2, scoreType=cv.ORB_HARRIS_SCORE, patchSize=31, fastThreshold=20):
  points_in_mask[i] = []
  kp_len[i] = []
  slices = {}
  correct_slices[i] = 0
  tp, tn, fp, fn = 0,0,0,0

  for idx, d in enumerate(data):
    kp = find_keypoints(d[0], nfeatures, scaleFactor, nlevels, edgeThreshold, firstLevel, WTA_K, scoreType, patchSize, fastThreshold)
    label = d[1]
    kp_len[i].append(len(kp))
    points_in_mask[i].append(0)
    in_mask = 0
    not_in_mask = 0
    for p in kp:
      [x,y] = p.pt
      if is_point_in_mask(label, int(x), int(y)):
        points_in_mask[i][idx] = int(points_in_mask[i][idx]) + 1
        slices[idx] = 1
        in_mask = 1
      else:
        not_in_mask = 1
    if in_mask == 1:
      tp = tp + 1
    elif in_mask == 0:
      fn = fn + 1

    if not_in_mask == 1:
      fp = fp + 1
    elif not_in_mask == 0:
      tn = tn + 1
    if in_mask == 1 and not_in_mask == 0:
      correct_slices[i] = correct_slices[i] + 1
  matrix[i] = {"tp": tp, "fn": fn, "fp": fp, "tn": tn}

In [ ]:
img_dir = "../data/preprocessed/images"
files = os.listdir(img_dir)
label_dir = "../data/preprocessed/labels"

data = []

for file in files:
  data.append(
    [cv.imread(os.path.join(img_dir, file), cv.IMREAD_GRAYSCALE),
     cv.imread(os.path.join(label_dir, file), cv.IMREAD_GRAYSCALE)]
  )
datalen = len(data)
t=6
parts = split_list(data, wanted_parts=t)

# nfeatures=500, scaleFactor=1.2, nlevels=8, edgeThreshold=31, firstLevel=0, WTA_K=2, scoreType=cv.HARRIS_SCORE, patchSize=31, fastThreshold=20
for nfeatures in [400, 500, 600]:
  for scaleFactor in np.arange(1.0, 1.11, 0.05):
    for nlevels in range(1, 6):
      for edgeThreshold in range(1, 41, 10):
        for firstLevel in range(0, 16, 3):
          for WTA_K in [2]:
            for scoreType in [cv.ORB_HARRIS_SCORE, cv.ORB_FAST_SCORE]:
              for patchSize in [31]:
                for fastThreshold in range(0, 41, 8):
                  print(f"nfeatures={nfeatures}, scaleFactor={scaleFactor}, nlevels={nlevels}, edgeThreshold={edgeThreshold}, firstLevel={firstLevel}, WTA_K={WTA_K}, scoreType={scoreType}, patchSize={patchSize}, fastThreshold={fastThreshold}")

                  threads = [None] * t
                  points_in_mask = [None] * t
                  kp_len = [None] * t
                  correct_slices = [None] * t
                  matrix = [None] * t
                  for i in range(len(threads)):
                      threads[i] = Thread(target=algo_run, args=(parts[i], points_in_mask, kp_len, correct_slices, matrix, i, nfeatures, scaleFactor, nlevels, edgeThreshold, firstLevel, WTA_K, scoreType, patchSize, fastThreshold))
                      threads[i].start()

                  for i in range(len(threads)):
                      threads[i].join()
                  points_in_mask_sum = sum([sum(sub) for sub in points_in_mask])
                  kp_len_sum=sum([sum(sub) for sub in kp_len])
                  avg_kp_len = kp_len_sum/datalen
                  
                  max_points_in_mask_value = max(max(points_in_mask))
                  max_kp_len_value = max(max(kp_len))
                  
                  min_points_in_mask_value = min(min(points_in_mask))
                  min_kp_len_value = min(min(kp_len))
                  
                  tp = sum([el["tp"] for el in matrix])
                  tn = sum([el["tn"] for el in matrix])
                  fp = sum([el["fp"] for el in matrix])
                  fn = sum([el["fn"] for el in matrix])
                  print(f"tp={tp}, tn={tn}, fp={fp}, fn={fn}")
                  sensitivity=tp/(tp+fn)
                  specificity=tn/(tn+fp)
                  g_mean=math.sqrt(sensitivity*specificity)
                  accuracy=(sensitivity+specificity)/2
                  print(f"sensitivity={sensitivity}, specificity={specificity}, acc={accuracy}, g-mean={g_mean}")

                  print(f"total points_in_mask={points_in_mask_sum}, total kp_len={kp_len_sum}")
                  print(f"avg points_in_mask={points_in_mask_sum/datalen}, avg kp_len={avg_kp_len}")
                  print(f"min points_in_mask={min_points_in_mask_value}, min kp_len={min_kp_len_value}")
                  print(f"max points_in_mask={max_points_in_mask_value}, max kp_len={max_kp_len_value}")
                  correct = sum(correct_slices)/datalen
                  print(f"correct_slices={correct}")
                  print("-------")
